# This notebook converts a trained Pytorch model to a TensorFlow version of the model.
More details can be found at https://github.com/onnx/tutorials/blob/master/tutorials/PytorchTensorflowMnist.ipynb

### Throughout the notebook, there are several spots where a file path or file name need to be specified for the processing of each model. Those locatations are inticated by '***'. Remove the *'s and instert the name without changing the file extension (.pth, .onnx, .pb). 

In [1]:
!wget -O examples/TooRareToWear_2.pth https://www.dropbox.com/s/3jqww0elemcl82i/TooRareToWear_2.pth?dl=1

--2019-11-22 13:37:36--  https://www.dropbox.com/s/3jqww0elemcl82i/TooRareToWear_2.pth?dl=1
Resolving www.dropbox.com... 162.125.8.1
Connecting to www.dropbox.com|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/3jqww0elemcl82i/TooRareToWear_2.pth [following]
--2019-11-22 13:37:36--  https://www.dropbox.com/s/dl/3jqww0elemcl82i/TooRareToWear_2.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce1d3ab5a120bf0da69fa86331d.dl.dropboxusercontent.com/cd/0/get/As2uAEHFIy1i3T6IonAb7hLaTK35EbOPKsqnqAqUMTq19hZTG4Dy-3k1pCe28OMapQ8eMB9LTOnY_tcHyMb9xTemvO0VTMrZlbcISW1D0GYjJfsrphH1is09NsoE6HjP8S0/file?dl=1# [following]
--2019-11-22 13:37:37--  https://uce1d3ab5a120bf0da69fa86331d.dl.dropboxusercontent.com/cd/0/get/As2uAEHFIy1i3T6IonAb7hLaTK35EbOPKsqnqAqUMTq19hZTG4Dy-3k1pCe28OMapQ8eMB9LTOnY_tcHyMb9xTemvO0VTMrZlbcISW1D0GYjJfsrphH1is09NsoE6HjP8S0/file?dl=1
Resolving uc

In [2]:
PYTORCH_MODEL = 'examples/TooRareToWear_2.pth'
ONNX_MODEL = 'examples/TooRareToWear_2.onnx'
TF_MODEL = 'examples/TooRareToWear_2.pb'

In [3]:
import torch
from fastai import vision as fv #installed via pip

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import onnx

from onnx_tf.backend import prepare #installed via pip

/Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/common/__init__.py:89: UserWarning: onnx_tf.common.get_outputs_names is deprecated. It will be removed in future release. Use TensorflowGraph.get_outputs_names instead.
  warnings.warn(message)
W1122 13:38:30.609096 4560811456 deprecation_wrapper.py:119] From /Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/handlers/backend/ceil.py:10: The name tf.ceil is deprecated. Please use tf.math.ceil instead.

W1122 13:38:30.616091 4560811456 deprecation_wrapper.py:119] From /Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/handlers/backend/depth_to_space.py:12: The name tf.depth_to_space is deprecated. Please use tf.compat.v1.depth_to_space instead.

W1122 13:38:30.619266 4560811456 deprecation_wrapper.py:119] From /Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/handlers/backend/erf.py:9: The name tf.erf is deprecated. Please use tf.math.er

### Saves the weights for the model/model custimization
Specify the path to the model in torch.load().

In [4]:
loc = torch.load(PYTORCH_MODEL, map_location=torch.device('cpu'))
body = fv.create_body(fv.models.resnet101, True, None)
data_classes = 2
nf = fv.callbacks.hooks.num_features_model(body) * 2
head = fv.create_head(nf, data_classes, None, ps=0.5, bn_final=False)
model = nn.Sequential(body, head)

### Load Model

In [5]:
trained_model = model.load_state_dict(loc['model'])

### Export the trained model to ONNX
In order to export the model, Pytorch exporter needs to run the model once and save this resulting trained model to a ONNX file. Therefore, we need to provide the input for the model we hope to convert. 

In [6]:
dummy_input = Variable(torch.randn(1, 3, 28, 28)) #(# of images, options for pixels should be one for b/w images and 3 for rgb images, x dimension, y diemsion) one rgb 28 x 28 picture will be the input to the model

In [7]:
torch.onnx.export(model, dummy_input, ONNX_MODEL, input_names=['input'], output_names=['output'])

### Load the ONNX file and import the model to Tensorflow

In [8]:
onnx_model = onnx.load(ONNX_MODEL)

tf_rep = prepare(onnx_model)

/Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op ConstantFill in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of ConvInteger in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:34: UserWarning: Fail to get since_version of DequantizeLinear in domain `` with max_inclusive_version=9. Set to 1.
  handler.ONNX_OP, handler.DOMAIN, version))
/Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-packages/onnx_tf/common/handler_helper.py:37: UserWarning: Unknown op GatherND in domain `ai.onnx`.
  handler.ONNX_OP, handler.DOMAIN or "ai.onnx"))
/Users/triznam/miniconda3/envs/tf_torch/lib/python3.7/site-pa

### Print statements to verify the model converted correctly 

In [9]:
# Input nodes to the model
print('inputs:', tf_rep.inputs)

# Output nodes from the model
print('outputs:', tf_rep.outputs)

# All nodes in the model
print('tensor_dict:')
print(tf_rep.tensor_dict)

inputs: ['input']
outputs: ['output']
tensor_dict:
{'0.0.weight': <tf.Tensor 'Const:0' shape=(64, 3, 7, 7) dtype=float32>, '0.1.bias': <tf.Tensor 'Const_1:0' shape=(64,) dtype=float32>, '0.1.num_batches_tracked': <tf.Tensor 'Const_2:0' shape=() dtype=int64>, '0.1.running_mean': <tf.Tensor 'Const_3:0' shape=(64,) dtype=float32>, '0.1.running_var': <tf.Tensor 'Const_4:0' shape=(64,) dtype=float32>, '0.1.weight': <tf.Tensor 'Const_5:0' shape=(64,) dtype=float32>, '0.4.0.bn1.bias': <tf.Tensor 'Const_6:0' shape=(64,) dtype=float32>, '0.4.0.bn1.num_batches_tracked': <tf.Tensor 'Const_7:0' shape=() dtype=int64>, '0.4.0.bn1.running_mean': <tf.Tensor 'Const_8:0' shape=(64,) dtype=float32>, '0.4.0.bn1.running_var': <tf.Tensor 'Const_9:0' shape=(64,) dtype=float32>, '0.4.0.bn1.weight': <tf.Tensor 'Const_10:0' shape=(64,) dtype=float32>, '0.4.0.bn2.bias': <tf.Tensor 'Const_11:0' shape=(64,) dtype=float32>, '0.4.0.bn2.num_batches_tracked': <tf.Tensor 'Const_12:0' shape=() dtype=int64>, '0.4.0.bn2.r

### Save the Tensorflow model to a file. 
You man rename the file, just keep the .pb file extenstion

In [10]:
tf_rep.export_graph(TF_MODEL)